In [2]:
# Imports
import pandas as pd
import numpy as np
import scipy.stats as stats
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns

# Plot settings
plt.rcParams['figure.dpi'] = 300

Import data

In [1]:
Marthasterias = []
with open ("Data/Marthasterias.glacialis.genelist.bed") as f:
    for line in f:
        Marthasterias.append(line.strip().split())
        
M_data = pd.DataFrame(Marthasterias,
                      columns = ['Chromosome', 'Start', 'End', 'Name', 'Dot'])

Branchiostoma = []
with open ("Data/Branchiostoma.lanceolatum.genelist.bed") as f:
    for line in f:
        Branchiostoma.append(line.strip().split())
        
B_data = pd.DataFrame(Branchiostoma,
                      columns = ['Chromosome', 'Start', 'End', 'Name', 'Dot'])

MB_orthology = np.loadtxt("Data/Marthasterias.glacialis+Branchiostoma.lanceolatum.txt", dtype = "str")

NameError: name 'pd' is not defined

In [4]:
# Branchiostoma data: removing non-chromosomal values
B_data = B_data.loc[B_data['Chromosome'].str.contains('BFL_')]

In [6]:
M_MB_orthdict = dict(zip(MB_orthology[:, 1], MB_orthology[:, 0]))
B_MB_orthdict = dict(zip(MB_orthology[:, 2], MB_orthology[:, 0]))

In [7]:
M_MB_data = M_data.replace({"Name": M_MB_orthdict})
B_MB_data = B_data.replace({"Name": B_MB_orthdict})

In [8]:
M_MB_data["Ortholog"] = M_MB_data["Name"].apply(lambda x:1 if 'ortholog' in x.lower() else 0)
B_MB_data["Ortholog"] = B_MB_data["Name"].apply(lambda x:1 if 'ortholog' in x.lower() else 0)

In [9]:
M_MB_ortho = M_MB_data.loc[M_MB_data['Ortholog'] == 1]
M_MB_dict = dict(zip(M_MB_ortho.Name, M_MB_ortho.Chromosome))

B_MB_ortho = B_MB_data.loc[B_MB_data['Ortholog'] == 1]
B_MB_dict = dict(zip(B_MB_ortho.Name, B_MB_ortho.Chromosome))

In [10]:
MB_Odata = pd.DataFrame({'Orthologs' : MB_orthology[:, 0],
                        'M' : MB_orthology[:, 0],
                        'B' : MB_orthology[:, 0]})

# Replace location in A and H with orthology dictionary keys
MB_Odata['M'] = MB_Odata['M'].map(M_MB_dict)
MB_Odata['B'] = MB_Odata['B'].map(B_MB_dict)

# Calculate number of orthologs for each pair of chromosomes
MB_Odata = MB_Odata.groupby(['M', 'B']).count().reset_index()

In [11]:
def hypertest(chrA, chrB, dataset, speciesA, speciesB, speciesAortho, speciesBortho):
    """
    inputs:
    chrA: species A chromosome name 
    chrB: species B chromosome name
    speciesA: column name in dataset for species A
    speciesB: column name in dataset for species B
    speciesAortho: orthology data for species A
    speciesBortho: orthology data for species B
    """
    
    M = len(list(set(dataset.Name.values.tolist()) & set(dataset.Name.values.tolist())))
    
    nA = dataset.loc[(dataset[speciesA] == chrA), 'Orthologs'].sum()
    nB = dataset.loc[(dataset[speciesB] == chrB), 'Orthologs'].sum()
    x = dataset.loc[(dataset[speciesA] == chrA) & (dataset[speciesB] == chrB), 'Orthologs'].sum()
    
    p = stats.hypergeom.sf(x - 1, M, nA, nB)
     
    return p

In [13]:
MB_Odata

,M,B,Orthologs


In [12]:
# MB
MB_Odata['p'] = MB_Odata.apply(lambda x: hypertest(x['M'], x['B'], dataset = MB_Odata, 
                                                   speciesA = 'M', speciesB = 'B',
                                                   speciesAortho = 'M_MB_data', speciesBortho = 'B_MB_data'), axis = 1)

# Apply BH testing correction
MB_Odata['result'], MB_Odata['p'] = pg.multicomp(MB_Odata['p'], method = 'fdr_bh') 

# Remove all rows that have been rejected in BH correction
MB_cOdata = MB_Odata.loc[MB_Odata["result"] == True]

ValueError: Expected a 1D array, got an array with shape (0, 3)